In [1]:
import stanza, re, benepar, psutil, gc, json, csv, time, string, os

import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns

from textstat import textstat
from datetime import datetime
from collections import Counter

import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk import Tree
from textblob import Word
from spellchecker import SpellChecker

import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, LogFormatter

In [ ]:
# These only need to run once
# stanza.download('en')
spacy.cli.download("en_core_web_sm")
nltk.download('punkt_tab')
benepar.download('benepar_en3')

In [6]:
source_texts= list()
source_texts.append("Kazuo Ishiguro - Never Let Me Go")
source_texts.append("Kazuo Ishiguro - The Remains of the Day")
source_texts.append("Kazuo Ishiguro - A Pale View of Hills-Knopf Doubleday Publishing Group (1990)")
source_texts.append("Kazuo Ishiguro - Never Let Me Go")
source_texts.append("Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_")
source_texts.append("The Buried Giant (Kazuo Ishiguro) (Z-Library)-1")
source_texts.append("Kazuo Ishiguro - The Unconsoled-Vintage (1996)")
directory_path = "C:/Users/Roland/Documents/AI/stylometry/"


Select choice of text by specifying index (starting with 0):

In [3]:
text_choice = 0

In [7]:
file_path = directory_path+source_texts[text_choice]+".txt"
print (file_path)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - Never Let Me Go.txt


Try reading the file with different encodings:

In [8]:
encodings = ['utf-8', 'ascii', 'latin-1','windows-1252', 'iso-8859-1', 'iso-8859-15']
texts = []
for i, encoding in enumerate(encodings):
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            test_text = file.read()
        print("""Successfully read the file with encoding: {encoding} (encoding index {ind}).  This gives the beginning as:\n{content}\n""".format(encoding=encoding, ind=i, content=test_text[0:200].replace("\n", "  ")))
        texts.append(test_text)
    except UnicodeDecodeError:
        print(f"Failed to read with encoding: {encoding}")

Failed to read with encoding: utf-8
Failed to read with encoding: ascii
Successfully read the file with encoding: latin-1 (encoding index 2).  This gives the beginning as:
My name is Kathy H. IÕm thirty-one years old, and IÕve been a carer now for over eleven years. That sounds long enough, I know, but actually they want me to go on for another eight months, until the e

Failed to read with encoding: windows-1252
Successfully read the file with encoding: iso-8859-1 (encoding index 4).  This gives the beginning as:
My name is Kathy H. IÕm thirty-one years old, and IÕve been a carer now for over eleven years. That sounds long enough, I know, but actually they want me to go on for another eight months, until the e

Successfully read the file with encoding: iso-8859-15 (encoding index 5).  This gives the beginning as:
My name is Kathy H. IÕm thirty-one years old, and IÕve been a carer now for over eleven years. That sounds long enough, I know, but actually they want me to go on for another

Are they the same?

In [11]:
for i in range(len(texts)):
    print (f"text {i} length {len(texts[i])}:-")
    for j, t in enumerate(texts):
        print (j, t == texts[i])

text 0 length 514835:-
0 True
1 True
2 True
text 1 length 514835:-
0 True
1 True
2 True
text 2 length 514835:-
0 True
1 True
2 True


Now load text file with selected encoding

In [ ]:
encoding = encodings[2]
with open(file_path, 'r', encoding=encoding) as file:
    text = file.read()
print(f"Total characters in book:  {len(text)}")

In [14]:
def check_for_file(file_path):
    if os.path.isfile(file_path):
        print("The file exists.")
    else:
        print("The file does not exist.")
    


In [12]:
save_path = file_path[:-4]+" - edited.txt"
print(save_path)


C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - Never Let Me Go - edited.txt


In [16]:
check_for_file(save_path)

The file does not exist.


In [17]:
for t in texts:
    with open(save_path, 'w', encoding='utf-8') as file:
        file.write(t)
    with open(save_path, 'r', encoding='utf-8') as file:
        text_edited = file.read()
    print(text_edited == t)

True
True
True


In [ ]:
text == text_edited

Watch out for this character as it is very awkward;

In [ ]:
print("first print")
print(ord("\x8e"))
print("second print")
print(chr(142))

Identify invalid characters using string:

In [ ]:

# Define a string containing only valid characters (alphanumeric + punctuation)
valid_chars = string.ascii_letters + string.digits + string.punctuation + " " = "\n"

# Example text with some invalid characters
#test_text = "This is an example text with invalid characters like Ò, ë, etc."
test_text = text
# Step 1: Identify invalid characters
invalid_chars = set([char for char in test_text if char not in valid_chars])

# Display the invalid characters found
print(f"Invalid characters found: {invalid_chars}")

# Step 2: Create a dictionary or list of tuples for replacement
# For simplicity, let's replace invalid characters with '?'
replacement_dict = {char: '?' for char in invalid_chars}

# If you want to create a list of tuples instead:
replacement_list = [(char, '?') for char in invalid_chars]

# Step 3: Replace invalid characters in the text
corrected_text = test_text
for char, replacement in replacement_dict.items():
    corrected_text = corrected_text.replace(char, replacement)

# Display the corrected text
#print(f"Corrected text: {corrected_text}")

# Output
print("Replacement dictionary:", replacement_dict)
print("Replacement list of tuples:", replacement_list)


Identify invalid characters using regular expression (re):
Don't use this one as the later one is better.

In [19]:
# Define the pattern for valid characters (alphanumeric, spaces, and basic punctuation)
valid_char_pattern = re.compile(r'[a-zA-Z0-9\s.:;,?!\'"()£\-&]')

valid_encodings = [2, 4, 5]
re_chars = len(valid_encodings) * [0]

for enc, encoding_index in enumerate(valid_encodings):
    test_text = texts[enc]

# Find invalid characters
    invalid_chars = set()
    for i, char in enumerate(test_text):
        if not valid_char_pattern.match(char):
            invalid_chars.add(char)

# Output the list of invalid characters and their positions
    invalid_char_positions = [(char, [i for i, c in enumerate(test_text) if c == char]) for char in invalid_chars]
    print(f"invalid characters using re for {encodings[encoding_index]}: {invalid_chars}")
#print(invalid_char_positions)
    re_chars[enc] = invalid_chars

invalid characters using re for latin-1: {'É', 'Ñ', 'Ó', 'Ô', '\x90', 'Ò', 'Õ', '\x88', '\x8e'}
invalid characters using re for iso-8859-1: {'É', 'Ñ', 'Ó', 'Ô', '\x90', 'Ò', 'Õ', '\x88', '\x8e'}
invalid characters using re for iso-8859-15: {'É', 'Ñ', 'Ó', 'Ô', '\x90', 'Ò', 'Õ', '\x88', '\x8e'}


In [21]:
# Define valid characters using the string module
valid_chars = string.ascii_letters + string.digits + string.punctuation + "àêé \n\t"
valid_chars = ''.join([char for char in valid_chars if char not in "{}~$%"])
print (f"valid characters used by string: {string.ascii_letters + string.digits + ' ' + string.punctuation}")
# Example text
#test_text = "This is a tÈst string with invàlid chàracters."
valid_encodings = [2, 4, 5]
string_chars = len(valid_encodings) * [0]

for enc, encoding_index in enumerate(valid_encodings):
    test_text = texts[enc]

    invalid_chars = set()
    for i, char in enumerate(test_text):
        if char not in valid_chars:
            invalid_chars.add(char)
    #invalid_chars.difference_update({"\n","\t"})

    invalid_char_positions = [(char, [i for i, c in enumerate(test_text) if c == char]) for char in invalid_chars]
    print(f"invalid characters found using string for {encodings[encoding_index]}: {invalid_chars}")
#print(invalid_char_positions)
    string_chars[enc] = invalid_chars



valid characters used by string: abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
invalid characters found using string for latin-1: {'É', 'Ñ', 'Ó', 'Ô', '\x90', 'Ò', 'Õ', '\x88', '\x8e'}
invalid characters found using string for iso-8859-1: {'É', 'Ñ', 'Ó', 'Ô', '\x90', 'Ò', 'Õ', '\x88', '\x8e'}
invalid characters found using string for iso-8859-15: {'É', 'Ñ', 'Ó', 'Ô', '\x90', 'Ò', 'Õ', '\x88', '\x8e'}


In [28]:
for char in invalid_chars:
    print(ord(char)," ", char)


201   É
209   Ñ
211   Ó
212   Ô
144   
210   Ò
213   Õ
136   
142   


In [ ]:
#just a little test:
''.join([char for char in "abcdefgh" if char not in "cd"])

In [ ]:
print(valid_chars)
print(ord(valid_chars[68]))  #this is the ' after the & in the list
print(ord(valid_chars[-13])) #this is the ` before the { in the list
print(ord("'"))
print(ord('"'))
print(ord("`"))

In [22]:
def display_chars(chars, method, unprintable = 142):
    number = len(chars)
    invalid_chars = '  '.join(char for char in chars)
    unprintable_message = " + unprintable character" if chr(unprintable) in invalid_chars else ""
    print(f"{number} invalid chars found using {method}:  {invalid_chars}{unprintable_message}")

In [23]:
for enc, encoding_index in enumerate(valid_encodings):
    print(f"with encoding {encodings[encoding_index]}")
    
    display_chars(re_chars[enc], "re   ")
    display_chars(string_chars[enc], "string")
        
    re_only = re_chars[enc] - string_chars[enc]
    string_only = string_chars[enc] - re_chars[enc]
    
    print(f"\n{len(re_only)} chars in re but not in string:    {'  '.join(char for char in re_only)}")
    print(f"{len(string_only)} chars in string but not in re:    {'  '.join(char for char in string_only)}\n")

with encoding latin-1
9 invalid chars found using re   :  É  Ñ  Ó  Ô    Ò  Õ     + unprintable character
9 invalid chars found using string:  É  Ñ  Ó  Ô    Ò  Õ     + unprintable character

0 chars in re but not in string:    
0 chars in string but not in re:    

with encoding iso-8859-1
9 invalid chars found using re   :  É  Ñ  Ó  Ô    Ò  Õ     + unprintable character
9 invalid chars found using string:  É  Ñ  Ó  Ô    Ò  Õ     + unprintable character

0 chars in re but not in string:    
0 chars in string but not in re:    

with encoding iso-8859-15
9 invalid chars found using re   :  É  Ñ  Ó  Ô    Ò  Õ     + unprintable character
9 invalid chars found using string:  É  Ñ  Ó  Ô    Ò  Õ     + unprintable character

0 chars in re but not in string:    
0 chars in string but not in re:    



In [ ]:
print(ord("é"))
a = "eé".encode('utf-8')
print(len(a))

In [24]:
def compare_texts(text1, text2):
    if len(text1) != len(text2):
        raise ValueError("Strings must be of the same length!")

# Create a list of tuples with differing characters
    differences = [(char1, char2) for char1, char2 in zip(text1, text2) if char1 != char2]

# Print the list of tuples
    print(differences[:100])

In [ ]:
compare_texts(texts[2], texts[3])

Now we are going to display all the invalid characters in their context to decide what to do about them. We will just do it for one of the versions as they are all the same.

In [30]:
def display_invalid_chars(test_text, window_size = 200, max_to_display = 8):
# Find all invalid characters and their positions
    #valid_chars = string.ascii_letters + string.digits + string.punctuation + " ()£-&\n\t"
    #print(valid_chars)
    invalid_char_positions = [(i, char) for i, char in enumerate(test_text) if char not in valid_chars]
                          
    icp_list_sorted = sorted(invalid_char_positions,key=lambda x: x[1])
    
    for pos, char in icp_list_sorted:
        start = max(pos - window_size // 2, 0)
        end = min(pos + window_size // 2, len(test_text))
        window = test_text[start:end]
    
    # Highlight the invalid character in the window
        highlighted_window = window[:pos-start] + f"[{char}]" + window[pos-start+1:]
        highlighted_window = highlighted_window.replace("\n", " ")
        highlighted_window = highlighted_window.replace("\t", " ")
        print(f"Invalid character '{char}' ({ord(char)}) at position {pos}:\n{highlighted_window}\n")




In [31]:
display_invalid_chars(texts[0])

Invalid character '' (136) at position 280718:
fore I realised Ruth and Chrissie were standing together at the end of it, having some sort of tte-[]-tte. I wasnÕt sure what to do: I didnÕt want to interrupt, but it was time we were leaving and I 

Invalid character '' (142) at position 200732:
fing. These were classes where we had to role play various people weÕd find out thereÑwaiters in caf[]s, policemen and so on. The sessions always got us excited and worried all at the same time, so we 

Invalid character '' (142) at position 236682:
t to think. The veterans usually laughed when one showed up and flicked through it quickly in a blas[] way before throwing it aside, so we did the same. When Ruth and I were remembering all this a few 

Invalid character '' (142) at position 246716:
ed from. ThatÕs why, when you were out there yourself Ñin the towns, shopping centres, transport caf[]sÑyou kept an eye out for ÒpossiblesÓÑthe people who might have been the models for you and y

In [ ]:
display_invalid_chars(texts[1])

In [ ]:
display_invalid_chars(texts[2])

In [ ]:
display_invalid_chars(texts[3])

In [ ]:
print("€" in texts[3])
print(texts[3][246245])

In [ ]:
print(ord(texts[2][146834]))

In [52]:
# Function to find and replace specified strings in a text
def replace_strings_with_context(text, replacements, window=10, silent = False):
    """
    Replaces specified strings within a text and prints the context around each replacement.
    
    Parameters:
    - text: The original string where replacements are done.
    - replacements: A dictionary where keys are the substrings to find, and values are their replacements.
    - window: Number of characters to show before and after the replaced string for context.
    
    Returns:
    - A modified string with the replacements applied.
    """
    counter = 0
    for old, new in replacements.items():
        index = text.find(old)
        while index != -1:
            # Extract context around the found string
            start = max(index - window, 0)
            end = min(index + len(old) + window, len(text))
            before = text[start:index]
            after = text[index + len(old):end]
            before = before.replace("\n", " ")
            before = before.replace("\t", " ")
            after = after.replace("\n", " ")
            after = after.replace("\t", " ")

            # Print before, replaced, and after strings
            if not silent:
                print(f"Before: {before}[{old}]{after}")
                print(f"After:  {before}[{new}]{after}\n")

            # Replace the string in the text
            text = text[:index] + new + text[index + len(old):]
            counter += 1
            
            # Find the next occurrence of the old string
            index = text.find(old, index + len(new))

    print("made {counter} changes")
    return text




In [37]:
print(ord("b"))
print(chr(98))
print(ord("Ž"))

98
b
381


In [54]:
replacements = dict()
# replacements["t}:le"] = "the"
# replacements["up o~ the"] = "up on the"
# replacements["the ~ort"] = "the sort"
# replacements["than ~ might"] = "than might"
# replacements["the house~"] = "the house."
# replacements["while ~earlier"] = "while earlier"
# replacements["meetings ~once"] = "meetings once"
# replacements["Ž"] = "é"
# replacements["mean- it"] = "mean it"
# replacements["¤"] = "s"
# replacements["Ð"] = "-"
replacements["É"] = '-'
replacements["Ó"] = '"'
replacements["Ò"] = '"'
replacements["Ô"] = '"'
# replacements["Rover.Ó I thanked"] = "Rover  I thanked"
# replacements['Rover." I thanked'] = "Rover  I thanked"
replacements["Õ"] = "'"
replacements[chr(136)] = "à"
replacements[chr(142)] = "é"
replacements[chr(144)] = "ê"
replacements["Ñ"] = "-"

for key, value in replacements.items():
        print(f"{key}: {value}")

É: -
Ó: "
Ò: "
Ô: "
Õ: '
: à
: é
: ê
Ñ: -


In [55]:
text_edited = replace_strings_with_context(texts[1],replacements, silent = True)

made {counter} changes


Now see if there are any problems left:

In [56]:
display_invalid_chars(text_edited)


Invalid character 'à' (224) at position 280718:
fore I realised Ruth and Chrissie were standing together at the end of it, having some sort of tête-[à]-tête. I wasn't sure what to do: I didn't want to interrupt, but it was time we were leaving and I 

Invalid character 'ê' (234) at position 280714:
- before I realised Ruth and Chrissie were standing together at the end of it, having some sort of t[ê]te-à-tête. I wasn't sure what to do: I didn't want to interrupt, but it was time we were leaving an

Invalid character 'ê' (234) at position 280721:
e I realised Ruth and Chrissie were standing together at the end of it, having some sort of tête-à-t[ê]te. I wasn't sure what to do: I didn't want to interrupt, but it was time we were leaving and I did



In [59]:
def display_text_in_context(source_text,target_text,window = 50):
    ind = source_text.find(target_text)
    if ind != -1:
        window = source_text[ind - window: ind + window + len(target_text)]
        window = window.replace("\n", " ")
        window = window.replace("\t", " ")
        print(window)
    else:
        print("not found")


In [45]:
for i in valid_encodings: print(encodings[i])

latin-1
iso-8859-1
iso-8859-15


In [62]:
print(text_edited[:1000])

My name is Kathy H. I'm thirty-one years old, and I've been a carer now for over eleven years. That sounds long enough, I know, but actually they want me to go on for another eight months, until the end of this year. That'll make it almost exactly twelve years. Now I know my being a carer so long isn't necessarily because they think I'm fantastic at what I do. There are some really good carers who've been told to stop after just two or three years. And I can think of one carer at least who went on for all of fourteen years despite being a complete waste of space. So I'm not trying to boast. But then I do know for a fact they've been pleased with my work, and by and large, I have too. My donors have always tended to do much better than expected. Their recovery times have been impressive, and hardly any of them have been classified as "agitated," even before fourth donation. Okay, maybe I am boasting now. But it means a lot to me, being able to do my work well, especially that bit about 

In [61]:
print(text_edited[-1000:])

 carrier bags. That was the only time, as I stood there, looking at that strange rubbish, feeling the wind coming across those empty fields, that I started to imagine just a little fantasy thing, because this was Norfolk after all, and it was only a couple of weeks since I'd lost him. I was thinking about the rubbish, the flapping plastic in the branches, the shore-line of odd stuff caught along the fencing, and I half-closed my eyes and imagined this was the spot where everything I'd ever lost since my childhood had washed up, and I was now standing here in front of it, and if I waited long enough, a tiny figure would appear on the horizon across the field, and gradually get larger until I'd see it was Tommy, and he'd wave, maybe even call. The fantasy never got beyond that - I didn't let it - and though the tears rolled down my face, I wasn't sobbing or out of control. I just waited a bit, then turned back to the car, to drive off to wherever it was I was supposed to be.














In [60]:
test_text.find("Chapter")

-1

Seems like there are no spurious headings to remove.

In [ ]:
test_text=text_edited
chapters = list()
for title in ["Prologue", "One", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine", "Ten", "Eleven"]:
    ind = test_text.find(title)
    while ind != -1:
        start = test_text[ind:ind + 100]
        start = start.replace("\n", " ")
        start = start.replace("\t", " ")
        print (start)
        chapters.append(title)
        ind = test_text.find(title, ind + 1)     
print(chapters)

In [ ]:
titles = list()
titles.append(("Prologue", "IT"))
titles.append(("Day One", "TONIGHT"))
titles.append(("Day Two", "STRANGE"))
titles.append(("Day Three - Morning", "LODGED"))
titles.append(("Day Three - Evening", "FEEL"))
titles.append(("Day Four", "HAVE"))
titles.append(("Day Six", "THIS"))

for edit_start, edit_end in titles:
    print(edit_start, edit_end)


In [ ]:
for edit_start, edit_end in titles:
    
    ind1 = text_edited.find(edit_start)
    ind2 = text_edited.find(edit_end)
    if ind2 > ind1 and ind2 < ind1 + 100 and ind1 != -1 and ind2 != -1:
        print (f"removing {edit_start} to {edit_end}")
        text_edited = text_edited[:ind1] + text_edited[ind2:]
    else:
        print(f"{edit_start} to {edit_end} not removed")


In [63]:
def find_capitalized_words(text):
    """
    Finds all words in the string that are in capitals, excluding words of one character.
    
    Parameters:
    - text: The input string.
    
    Returns:
    - A list of fully capitalized words with more than one character.
    """
    # Use regex to match words with all capital letters and exclude single-character words
    capitalized_words = re.findall(r'\b[A-Z]{2,}\b', text)
    return capitalized_words


In [64]:
capitalized_words = find_capitalized_words(texts[0])
for word in capitalized_words:
    if word != "OK":
        ind = text_edited.find(word)
        window = text_edited[ind-75:ind+40]
        window = window.replace("\n", " ")
        window = window.replace("\t", " ")
        print(window)


gan to notice all kinds of other things the veteran couples had taken from TV programmes: the way they gestured to 


In [ ]:
def display_text_in_context(source_text,target_text,window = 50):
    ind = source_text.find(target_text)
    if ind != -1:
        window = source_text[ind - window: ind + window + len(target_text)]
        window = window.replace("\n", " ")
        window = window.replace("\t", " ")
        print(window)
    else:
        print("not found")


In [ ]:
display_text_in_context(text_edited, "been given such a privilege")

In [ ]:
print(text_edited[:20000])

In [67]:
save_path = save_path[:-4]+" 1.txt"
print(save_path)




C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - Never Let Me Go - edited 1.txt


In [69]:
with open(save_path, 'w', encoding='utf-8') as f:
    f.write(text_edited)

Check that saving and loading the corrected file doesn't change it.  There will be a change when it encounters é because it takes 2 bytes.  Up to that point, there should be no differences.

In [74]:
with open(save_path) as f:
    text = f.read()

In [75]:
text == text_edited

False

In [ ]:
save_path = save_path[:-5]
save_path = save_path + " 1.txt"
print(save_path)

In [76]:
text_edited.find("é")

200732

In [82]:
print(text[:200732] == text_edited[:200732])
print(len(text))
print(len(text_edited))

True
514847
514835


In [80]:
print(text[:200733] == text_edited[:200733])
print(len(text))
print(len(text_edited))

False
514847
514835


In [ ]:

# Sample text with hyphenated words across lines
text = texts[3]
# Use regex to find words that are hyphenated across lines
# Match sequences where a hyphen is at the end of a line, followed by a newline, and then continued with a word
hyphenated_words = re.findall(r"(\w+)-\n(\w+)", text)

# Print the hyphenated words
for first_part, second_part in hyphenated_words:
    print(f"Found broken word: {first_part}-{second_part}")
print(len(hyphenated_words))
print(len(text))
